In [10]:
import datetime
import math
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from requests import Request, Session
import json
import nltk
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
import requests
from bs4 import BeautifulSoup
import cryptocompare


googlePrin = 'news.google.com'
googleFinal = '&hl=en'

bingPrin = 'www.bing.com/news'
bingFinal = '&hl=en&qft=sortbydate%3d"1"&form=YFNR'

import tweepy
TW_API_KEY = 'kvXdTL2YkKj6rAlQQy11gJMUe'
TW_API_SECRET_KEY = '04ry6xwn700PhENHKXdem13dN5bERNH0T1vBIIbMkRpiKyUH4N'
TW_ACCESS_TOKEN = '4872760203-qbVfsijOOnYX2g55q71NbWeULjyIpsQ5y8lmSOY'
TW_ACCESS_TOKEN_SECRET = '2cKaYgh7JBiIL6NNMlywJFbyMGTJDYIQPz9iqBAiXT024'
auth = tweepy.OAuthHandler(TW_API_KEY, TW_API_SECRET_KEY)
auth.set_access_token(TW_ACCESS_TOKEN, TW_ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
sia = SIA()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')

stop_words = stopwords.words('spanish')


def buscarEnTwitter(unaCrypto):
    status = api.rate_limit_status()
    misTweets = set()
    if status['resources']['search']['/search/tweets']['remaining']:
        someTweets = [status._json for status in tweepy.Cursor(api.search_tweets, q=unaCrypto, count=180, tweet_mode='extended', lang='en').items(180)]
        for aTweet in someTweets:
            misTweets.add(aTweet["full_text"])   
    return misTweets
    

def buscarEn(prin, fin, unaCrypto):
    headers = {
    "User-Agent":
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.19582"
    }
    url =  'https://' + prin + '/search?q=' + unaCrypto + fin
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.text, 'lxml')
    unasNoticias = set()
    for resultado in soup.select('.card-with-cluster'):    
        titulo = resultado.select_one('.title').text
        unasNoticias.add(titulo)
    unasNoticias = crearListaSinRepetidos(unasNoticias) 
    return unasNoticias


def buscarUnaCrypto(crypto):
    unasNoticias = set()
    unasNoticias = unasNoticias.union(buscarEnTwitter(crypto))
    unasNoticias = unasNoticias.union(buscarEn(googlePrin, googleFinal, crypto))
    unasNoticias = unasNoticias.union(buscarEn(bingPrin, bingFinal, crypto))
    return unasNoticias

def procesarTexto(unasNoticias):
    tokens = []
    for unaNoticia in unasNoticias:
        toks = tokenizer.tokenize(unaNoticia)
        toks = [t.lower() for t in toks if t.lower() not in stop_words]
        tokens.extend(toks)
    return tokens 

def sentimientosDe(crypto):
    noticiasAux = buscarUnaCrypto(crypto)
    cryptoAux = analizarSentimiento(noticiasAux, crypto)
   

def analizarSentimiento(unasNoticias, crypto):
    palabrasClave = []
    positivas = 0
    negativas = 0
    neutras = 0
    noticias = 0

    archivo = open("analisis.txt", "a")
    archivo.write(crypto)
    archivo.write(": ")
    archivo.write("\n")
    
    spacy.load('en_core_web_sm')
    for unaNoticia in unasNoticias:
        noticias += 1
        nlp = spacy.load('en_core_web_sm')
        nlp.add_pipe('spacytextblob')
        doc = nlp(unaNoticia)
        polarity = doc._.blob.polarity
        clave = doc._.blob.sentiment_assessments.assessments
        #print(doc._.blob.sentiment_assessments.assessments)
        
        if clave != []:
            palabrasClave.append(clave)
            archivo.write(pasarAString(clave))
            archivo.write("\n")

        if polarity > 0.2:
            positivas += 1
        elif polarity < -0.2:
            negativas += 1
        else:
            neutras += 1
    cantPalabrasClave = len(palabrasClave)
    archivo.write('Cantidad de noticias: '+str(noticias)+'\n')
    archivo.write('Resultados positivos: '+str(positivas)+'\n')
    archivo.write('Resultados negativos: '+str(negativas)+'\n')
    archivo.write('Resultados neutros: '+str(neutras)+'\n\n')
    archivo.write("Porcentaje de resultados positivos: " + str((positivas*100)/cantPalabrasClave) + "%"+'\n')
    archivo.write("Porcentaje de resultados neutrales: " + str((neutras*100)/cantPalabrasClave)+ "%"+'\n')
    archivo.write("Porcentaje de resultados negativos: " + str((negativas*100)/cantPalabrasClave) + "%"+'\n\n')
    print('Crypto:', crypto)
    print('Cantidad de noticias:', noticias)
    print('Resultados positivos:', positivas)
    print('Resultados negativos:',negativas)
    print('Resultados neutros:',neutras)
    print("Porcentaje de resultados positivos: " + str((positivas*100)/cantPalabrasClave) + "%")
    print("Porcentaje de resultados neutrales: " + str((neutras*100)/cantPalabrasClave) + "%")
    print("Porcentaje de resultados negativos: " + str((negativas*100)/cantPalabrasClave) + "%")
    
    archivo.close()

def crearListaSinRepetidos(lista):
    result = []
    for item in lista:
        if item not in result:
            result.append(item)
    return result

def pasarAString(lista):
    string='['
    
    for tupla in lista:
        string+='('+convertirAStr(tupla[0])
        string+=', '+str(tupla[1])
        string+=', '+str(tupla[2])
        string+=', '+str(tupla[3])+'), '
    string+=']'    
    return string

def convertirAStr(lista):
    string='['
    for item in lista:
        string+=item+', '
    string+=']'
    return string

def precioEnDeterminadaFecha(crypto, dia, mes, anio):
    valor = 0;
    valor = cryptocompare.get_historical_price(crypto, 'EUR', datetime.datetime(anio,mes,dia))
    archivo = open("analisis.txt", "a")
    archivo.write('Valor de la moneda el dia '+str(dia) +'/'+str(mes)+'/'+ str(anio)+ ' es: '+str(valor))
    print('Valor de la moneda el dia '+str(dia) +'/'+str(mes)+'/'+ str(anio)+ ' es: '+str(valor))

moneda= 'BTC'
sentimientosDe(moneda)
precioEnDeterminadaFecha(moneda, 4, 11, 2022)

Crypto: BTC
Cantidad de noticias: 177
Resultados positivos: 47
Resultados negativos: 5
Resultados neutros: 125
Porcentaje de resultados positivos: 42.34234234234234%
Porcentaje de resultados neutrales: 112.61261261261261%
Porcentaje de resultados negativos: 4.504504504504505%
Valor de la moneda el dia 4/11/2022 es: {'BTC': {'EUR': 21122.34}}
